In [5]:
## Transforma arquivos para modelagem geral de tópicos

import re

dataset = "Zookeeper"
file = open("ground_truths/" + dataset + ".txt", "r")
LINE_PATTERN = "#[0-9]+#"
SUMMARY_PATTERN = "#summary:#"
line_file = []
summary_file = []
line_match = True
summary_match = True
line_count = 0

for line in file:

        if len(line) == 1:                  
            summary_match = False
            line_match = False
            continue
        elif re.match(LINE_PATTERN, line):            
            line_match = True
            continue
        elif re.match(SUMMARY_PATTERN, line):
            summary_match = True
            continue
        elif (line_match):
            line_file.append(line)
            line_count += 1
            continue
        elif (summary_match):
            for elem in range(line_count):                
                summary_file.append(line.rstrip() + '\n')
            line_count = 0   
            continue

with open ("ground_truths/" + dataset + "_lines.txt", "w") as f:
     for line in line_file:
          f.write(f"{line}")

with open ("ground_truths/" + dataset + "_summaries.txt", "w") as f:
     for line in summary_file:
          f.write(f"{line}")

In [6]:
## Transforma arquivos para modelagem local de tópicos, parseando antes

## Drain parameters

import DrainMethod
import os

## Step 1 - Log Parsing Using Drain

dataset = "Zookeeper"
input_dir = os.path.join(os.getcwd(), "ground_truths/") # The input directory of raw logs
log_format = '<Content>' # Format of the file, if there are different fields
output_dir = input_dir  # The output directory of parsing results
logName = dataset + "_lines.txt" # Name of file to be parsed
#file = open("ground_truths/" + dataset + "_lines.txt", "r")
regex = [] # Regex strings for Drain execution
depth = 5 # Max depth of the parsing tree
st = 0.6 # Drain similarity threshold


## Code

print('\n=== Starting Drain Parsing ===')
indir = os.path.join(input_dir, os.path.dirname(logName))
print(indir)
log_file = os.path.basename(logName)

parser = DrainMethod.LogParser(log_format=log_format, indir=indir, outdir=output_dir, rex=regex, depth=depth, st=st)
parser.parse(log_file)

parsedresult=os.path.join(output_dir, log_file + '_structured.csv')   


    



=== Starting Drain Parsing ===
c:\Users\vbert\OneDrive\DOUTORADO Poly Mtl\Projeto\CSL\CSL-1\ground_truths/
Parsing file: c:\Users\vbert\OneDrive\DOUTORADO Poly Mtl\Projeto\CSL\CSL-1\ground_truths/Zookeeper_lines.txt


Parsing Progress: 100%|██████████| 2020/2020 [00:00<00:00, 10177.94it/s]


Parsing done. [Time taken: 0:00:00.422869]


In [3]:
## Method to find the most representative line inside the cluster

from nltk.tokenize import WhitespaceTokenizer

tk = WhitespaceTokenizer()

## raw_lines = list of lines inside LogSummary's cluster
## word_list = list of tokens composed by the LDA/BertTopic
def find_best_line(raw_lines, word_list):
    closest_line = 0
    similar_tokens = 0
    max_similarity = 0
    for idx, line in enumerate(raw_lines):
        tokenized_line = tk.tokenize(line.lower())
        for token in tokenized_line:
            if token in word_list:
                similar_tokens += 1
        #print ("Line {} has {} identical tokens".format(idx, similar_tokens))
        if similar_tokens > max_similarity:
           max_similarity = similar_tokens
           closest_line = idx
        similar_tokens = 0
    print("The closest line is {}, with {} identical tokens to the topic".format(closest_line, max_similarity)) 
    print("Line {} is: {}".format(closest_line, raw_lines[closest_line]))   
    return (raw_lines[closest_line])        

#find_similar_lines(cluster_lines, 0, words)

In [12]:
## Testes com LDA

import gensim.corpora as corpora
import pandas as pd
import gensim

dataset = "Zookeeper"
csv = pd.read_csv("ground_truths/Zookeeper_lines.txt_structured.csv")
content = csv["EventTemplate"]
num_topics = 10
line_file = []
line_set = []

# Converts sentences to words
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

for idx, line in enumerate(content):
    line_set.append(line + '\n')

    if (idx % 20 == 19):
  
        # Converts to words
        data_words = list(sent_to_words(line_set))
        # Creates dictionary
        id2word = corpora.Dictionary(data_words)
        # Creates corpora
        corpus = [id2word.doc2bow(text) for text in data_words]
        # Builds LDA model
        lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics)
        # Gets word topics
        x = lda_model.show_topics(num_topics=1, num_words=10,formatted=False)
        topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

        #Below Code Prints Only Words 
        for topic,words in topics_words:
            summary =  " ".join(words)    

        # #Appends summary to general line file
        # for num in range(20):
        #     line_file.append(summary)

        #Finds most representative line inside the cluster
        best_line = find_best_line(line_set, summary)

        for num in range(20):
            line_file.append(summary)

        line_set = []

## Writes external file with created topics
with open ("ground_truths/" + dataset + "_lda_topics.txt", "w") as f:
     for line in line_file:
          f.write(f"{line}\n")

The closest line is 4, with 8 identical tokens to the topic
Line 4 is: WARN Interrupted while waiting for message on queue

The closest line is 6, with 6 identical tokens to the topic
Line 6 is: INFO Established session <*> with negotiated timeout <*> for client <*> : <*>

The closest line is 1, with 6 identical tokens to the topic
Line 1 is: WARN caught end of stream exception

The closest line is 0, with 9 identical tokens to the topic
Line 0 is: INFO Closed socket connection for client <*> : <*> which had sessionid <*>

The closest line is 0, with 6 identical tokens to the topic
Line 0 is: INFO Client attempting to establish new session at <*> : <*>

The closest line is 6, with 8 identical tokens to the topic
Line 6 is: INFO Established session <*> with negotiated timeout <*> for client <*> : <*>

The closest line is 2, with 6 identical tokens to the topic
Line 2 is: WARN Connection request from old client <*> : <*> ; will be dropped if server is in r-o mode

The closest line is 10,

In [4]:
## Testes com Bertopic

from sklearn_extra.cluster import KMedoids
from bertopic import BERTopic
import pandas as pd
import numpy as np
import pickle
import os


dataset = "hdfs"
csv = pd.read_csv("ground_truths/hdfs_lines.txt_structured.csv")
content = csv["EventTemplate"]
num_topics = 10
line_file = []
line_set = []
cluster_model = KMedoids(n_clusters = 1)
topic_model = BERTopic(hdbscan_model=cluster_model)

for idx, line in enumerate(content):

    line_set.append(line + '\n')

    if (idx % 20 == 19):
  
        print("Chegamos ao idx {}".format(idx))

        #Applies BertTopic
        topics, probs = topic_model.fit_transform(line_set)

        #Gets summary of topics
        topic_model.get_topic(0)
        top_topic = topic_model.get_topic(0)
        words = [i[0] for i in top_topic]
        summary = ' '.join(words)

        #Finds most representative line inside the cluster
        best_line = find_best_line(line_set, summary)

        for num in range(20):
            line_file.append(summary)

        line_set = []

## Writes external file with created topics
with open ("ground_truths/" + dataset + "_bert_topics.txt", "w") as f:
     for line in line_file:
          f.write(f"{line}\n")

Chegamos ao idx 19
The closest line is 0, with 9 identical tokens to the topic
Line 0 is: RAS APP FATAL ciod : Error creating node map from file /home/pakin1/comms-conc/xyzt-1x1x32768x1.map : Bad address

Chegamos ao idx 39
The closest line is 0, with 8 identical tokens to the topic
Line 0 is: RAS KERNEL INFO instruction cache parity error corrected

Chegamos ao idx 59
The closest line is 0, with 9 identical tokens to the topic
Line 0 is: RAS KERNEL INFO ciod : Message code 0 is not 51 or 4294967295

Chegamos ao idx 79
The closest line is 0, with 7 identical tokens to the topic
Line 0 is: RAS KERNEL FATAL data TLB error interrupt

Chegamos ao idx 99
The closest line is 0, with 7 identical tokens to the topic
Line 0 is: RAS KERNEL FATAL data TLB error interrupt

Chegamos ao idx 119
The closest line is 0, with 7 identical tokens to the topic
Line 0 is: RAS KERNEL FATAL data TLB error interrupt

Chegamos ao idx 139
The closest line is 0, with 7 identical tokens to the topic
Line 0 is: RAS